## МОДУЛЬ ВИЗУАЛИЗАЦИИ УГЛОВ ЭЙЛЕРА
---
МОДУЛЬ ПРЕДНАЗНАЧЕН ДЛЯ ВИЗУАЛИИЗАЦИИ ПОКАЗАНИЙ УГЛОВ ЭЙЛЕРА ПОЛУЧЕННЫХ ПО КАНАЛУ ГИРОСКОПА, АКСЕЛЕРОМЕТРА, КОМПЛИМЕНТАРНОГО ФИЛЬТРА

### Блок импорта библиотек необходимых для работы модуля

In [ ]:
# Подключить библиотеки для мат обработки данных и генерации случайных данных
import numpy as np
import math

# Подключить библиотеки для работы с датой/временем
from datetime import datetime
import time

# Подключить библиотеки для асинхронной работы
import asyncio

# Подключить библиотеки для работы с базой данных redis
import redis

# Подключить библиотеки для работы с JSON
import json

# Подключить библиотеки для работы с завершающими функциями Jupyter Lab
import atexit

# Подключить библиотеки для работы с виджетами в Jupyter Lab
import ipywidgets as widgets
from   ipywidgets import Output
from IPython.display import display

# -----------------------------------------------------------------------
# Подключить пользовательские модули
# -----------------------------------------------------------------------
# Подключить модуль для подключения к БД Redis
from py.lib.lib_work_redis_rev01_v01 import *
# Подключить модуль для создания и обновления HTML элементов отображения данных об углах Эйлера
from py.lib.lib_work_html_widget_rev01_v01 import *
# Подключить модуль с оберткой для синхронных функций в стиле setInterval JavaScript
from py.lib.lib_setinterval_rev01_v01 import *

### Блок настроек среды исполнения Jupyter lab

In [ ]:
# Инструкция требующая повторно перезагружать пользовательские функции после изменения их кода
%load_ext autoreload
%autoreload 2

### Блок объявления глобальных переменных модуля

In [ ]:
# Создать алиасы индексов соответствующих под-массивов, для удобства обращения
ROLL_IND    = 0
PITCH_IND   = 1
YAW_IND     = 2

# Глобальный массив, хранит углы Эйлера полученные с помощью акселерометра. является копией соответствующего массива класса ReadSensorIMU
AccArrAngle  = [[0,0,0],[0,0,0],[0,0,0]]
# Глобальный массив, хранит углы Эйлера полученные с помощью гироскопа. является копией соответствующего массива класса ReadSensorIMU
GyroArrAngle = [[0,0,0],[0,0,0],[0,0,0]]
# Глобальный массив, хранит углы Эйлера полученные с помощью фильтра слияния. является копией соответствующего массива класса ReadSensorIMU
CompArrAngle = [[0,0,0],[0,0,0],[0,0,0]]

### Блок синхронизации данных об углах Эйлера с базой данных

In [ ]:
ConnectDB =  ConnectDB() # выполнить подключение к базе данных Redis

In [ ]:
# Создать функцию для синхронизации данных углов эйлера хранящихся в БД и глобальными переменными модуля
def ReadEulerDate():
    global AccArrAngle, GyroArrAngle, CompArrAngle

    # Считать и десериализировать данные массива углов Эйлера из Redis по каналу Акселерометра
    AccArrAngle_json = ConnectDB.get('AccArrAngle')
    AccArrAngle = json.loads(AccArrAngle_json)

    # Считать и десериализировать данные массива углов Эйлера из Redis по каналу Гироскопа
    GyroArrAngle_json = ConnectDB.get('GyroArrAngle')
    GyroArrAngle = json.loads(GyroArrAngle_json)

    # Считать и десериализировать данные массива углов Эйлера из Redis по каналу Комплементарного фильтра
    CompArrAngle_json = ConnectDB.get('CompArrAngle')
    CompArrAngle = json.loads(CompArrAngle_json)



In [ ]:
# Создаем асинхронную обертку для синхронной функции, с целью выполнения ее в асинхронном режиме
async def AsyncReadEulerDate():
    AsyncFunc = SetInterval( 0.1, ReadEulerDate ) # период синхронизации 100 ms
    AsyncFunc.start()

await AsyncReadEulerDate()

### Блок визуализации углов Эйлера по каналу Акселерометра

In [ ]:
# Создать виджеты для отображения значений углов Эйлера по каналу Акселерометра
Roll_Acc, Pitch_Acc, Yaw_Acc, A_Acc, B_Acc = CreateHTMLwidgest()
# Отобразить виджеты
display(widgets.HBox([widgets.Label(value='Roll_Acc'), Roll_Acc, A_Acc, widgets.Label(value='Pitch_Acc'), Pitch_Acc, B_Acc, widgets.Label(value='Yaw_Acc'), Yaw_Acc]))

In [ ]:
# Создать асинхронную обертку кода для асинхронного выполнения кода функции обновления содержимого виджетов углов по каналу Акселерометра
async def AsyncAccEulerUpdateVal():
    AsyncFunc = SetInterval( 0.25, UpdateHTMLwidgest, Roll_Acc, Pitch_Acc, Yaw_Acc, AccArrAngle ) # период синхронизации 250 ms
    AsyncFunc.start()

await AsyncAccEulerUpdateVal()

In [ ]:
# Создаем асинхронную обертку кода для асинхронного выполнения кода функции
async def AsyncAccEulerUpdateVal():
    AsyncFunc = SetInterval( 0.3, AccEulerUpdateVal ) # период синхронизации 100 ms
    AsyncFunc.start()

await AsyncAccEulerUpdateVal()

### Блок визуализации углов Эйлера по каналу Гироскопа

In [ ]:
# Функция для обновления значений
def GyroEulerUpdateVal():
    #global AccArrAngle, GyroArrAngle, CompArrAngle
    Roll_Gyro.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[ROLL_IND][-1]:.1f}</span></div>'
    Pitch_Gyro.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[PITCH_IND][-1]:.1f}</span></div>'
    Yaw_Gyro.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[YAW_IND][-1]:.1f}</span></div>'
    A_Gyro.value = ''
    B_Gyro.value = ''

# Создание виджетов для отображения значений переменных
Roll_Gyro = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[ROLL_IND][-1]:.1f}</span></div>')
Pitch_Gyro = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[PITCH_IND][-1]:.1f}</span></div>')
Yaw_Gyro = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[YAW_IND][-1]:.1f}</span></div>')
A_Gyro = widgets.HTML(value='', layout=widgets.Layout(width='50px'))
B_Gyro = widgets.HTML(value='', layout=widgets.Layout(width='50px'))

# Отображение виджетов
display(widgets.HBox([widgets.Label(value='Roll_Gyro'), Roll_Gyro, A_Gyro, widgets.Label(value='Pitch_Gyro'), Pitch_Gyro, B_Gyro, widgets.Label(value='Yaw_Gyro'), Yaw_Gyro]))

# Создаем асинхронную обертку кода для асинхронного выполнения кода функции
async def AsyncGyroEulerUpdateVal():
    AsyncFunc = SetInterval( 0.3, GyroEulerUpdateVal ) # период синхронизации 100 ms
    AsyncFunc.start()

await AsyncGyroEulerUpdateVal()

### Блок визуализации углов Эйлера по каналу Комплиментарного фильтра

In [ ]:
# Функция для обновления значений
def CompEulerUpdateVal():
    #global AccArrAngle, GyroArrAngle, CompArrAngle
    Roll_Comp.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[ROLL_IND][-1]:.1f}</span></div>'
    Pitch_Comp.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[PITCH_IND][-1]:.1f}</span></div>'
    Yaw_Comp.value = f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 10px">{CompArrAngle[YAW_IND][-1]:.1f}</span></div>'
    A_Comp.value = ''
    B_Comp.value = ''

# Создание виджетов для отображения значений переменных
Roll_Comp = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[ROLL_IND][-1]:.2f}</span></div>')
Pitch_Comp = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[PITCH_IND][-1]:.2f}</span></div>')
Yaw_Comp = widgets.HTML(value=f'<div style="text-align: right; width: 100px; border: 3px solid #000"><span style="padding-right: 3px">{CompArrAngle[YAW_IND][-1]:.2f}</span></div>')
A_Comp = widgets.HTML(value='', layout=widgets.Layout(width='50px'))
B_Comp = widgets.HTML(value='', layout=widgets.Layout(width='50px'))

# Отображение виджетов
display(widgets.HBox([widgets.Label(value='Roll_Comp'), Roll_Comp, A_Comp, widgets.Label(value='Pitch_Comp'), Pitch_Comp, B_Comp, widgets.Label(value='Yaw_Comp'), Yaw_Comp]))

# Создаем асинхронную обертку кода для асинхронного выполнения кода функции
async def AsyncCompEulerUpdateVal():
    AsyncFunc = SetInterval( 0.3, CompEulerUpdateVal ) # период синхронизации 100 ms
    AsyncFunc.start()

await AsyncCompEulerUpdateVal()